In [2]:
import os

In [1]:
%pwd


'c:\\Users\\Suyash\\Desktop\\End_to_End_projects\\Text_Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Suyash\\Desktop\\End_to_End_projects\\Text_Summarizer'

In [5]:
# creating entities

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path


In [6]:
from textsummerizer.constants import *
from textsummerizer.utils.common import read_yaml, create_directory

In [7]:
# Configurations

class ModelEvaluationConfigManager:
    def __init__(
            self,
            config_file_path: CONFIG_FILE_PATH,
            params_file_path: PARAMS_FILE_PATH):
            
            self.config = read_yaml(config_file_path)
            self.params = read_yaml(params_file_path)

            create_directory([self.config.artifacts_root])

    def get_model_evalution_config(self) -> ModelEvaluationConfig:
          config = self.config.model_evaluation

          model_evaluation_config = ModelEvaluationConfig(
              root_dir = config.root_dir,
              data_path = config.data_path,
              model_path = config.model_path,
              tokenizer_path = config.tokenizer_path,
              metric_file_name = config.metric_file_name
          )

          return model_evaluation_config

    

In [8]:
%pwd

'c:\\Users\\Suyash\\Desktop\\End_to_End_projects\\Text_Summarizer'

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset, load_from_disk, load_metric
import torch
import pandas as pd
import tqdm

In [11]:
# components for model evaluation

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    # Evalution

def generate_batch_sized_chunks(list_of_elements, batch_size):
  """split the dataset into smaller batches that we can process simultaneously
  yield successive batch sized chunks from list_of_elements."""

  for i in range(0,len(list_of_elements),batch_size):
    yield list_of_elements[i:i+batch_size]

def calculate_metric_on_test_ds(dataset,metric,model,tokenizer,
                                batch_size=16,device="cuda" if torch.cuda.is_available() else "cpu",
                                column_text="article",
                                column_summary="highlights"):
  article_batches = list(generate_batch_sized_chunks(dataset[column_text],batch_size))
  target_batches = list(generate_batch_sized_chunks(dataset[column_summary],batch_size))

  for article_batch, target_batch in tqdm(zip(article_batches,target_batches), total=len(article_batches)):
    inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                       padding="max_length", return_tensor='pt')
    summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                               attention_mask=inputs["attention_mask"].to(device),
                               length_penalty=0.8, num_beams=8, max_length=128)
    '''parameter for length penalty ensures that the model does not generate sequences that are too long'''

    # Finally, we decode the generated texts,
    # replace the token, and the decoded texts with the references to the metric
    decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                          clean_up_tokenization_spaces=True)
    for s in summaries]

    decoded_summaries = [d.replace(""," ") for d in decoded_summaries]

    metric.add_batch(predictions=decoded_summaries,references=target_batch)

  # Finally compute and return the ROUGE scores
  score = metric.compute()
  return score

def eval(self):
   device = "cuda" if torch.cuda.is_available() else "cpu"
   tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
   model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

   dataset_samsum_pt = load_from_disk(self.config.data_path)

   rouge_names = ["rouge1","rouge2","rougeL","rougeLsum"]
   rouge_metric = load_metric('rouge')

   score = self.calculate_metric_on_test_ds(
      dataset_samsum_pt["test"],rouge_metric,model_pegasus,tokenizer,
      batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
   )

   rouge_dict = dict((rn,score[rn].mid.fmeasure) for rn in rouge_names)

   df = pd.DataFrame(rouge_dict,index=['pegasus'])
   df.to_save(self.config.metric_file_name,index=False)


In [ ]:
# model evaluation pipeline
try:
    config = ModelEvaluationConfigManager()
    model_evaluation_config = config.get_model_evalution_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.eval()
except Exception as e:
    raise e